In [1]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import struct
%matplotlib inline

In [2]:
class MNIST():
    def __init__(self, directory):
        self._directory = directory
        
        self._training_data = self._load_binaries("train-images-idx3-ubyte")
        self._training_labels = self._load_binaries("train-labels-idx1-ubyte")
        self._test_data = self._load_binaries("t10k-images-idx3-ubyte")
        self._test_labels = self._load_binaries("t10k-labels-idx1-ubyte")
        
        np.random.seed(0)
        samples_n = self._training_labels.shape[0]
        random_indices = np.random.choice(samples_n, samples_n // 10, replace = False)
        np.random.seed()
        
        self._validation_data = self._training_data[random_indices]
        self._validation_labels = self._training_labels[random_indices]
        self._training_data = np.delete(self._training_data, random_indices, axis = 0)
        self._training_labels = np.delete(self._training_labels, random_indices)
    
    def _load_binaries(self, file_name):
        path = os.path.join(self._directory, file_name)
        
        with open(path, 'rb') as fd:
            check, items_n = struct.unpack(">ii", fd.read(8))

            if "images" in file_name and check == 2051:
                height, width = struct.unpack(">II", fd.read(8))
                images = np.fromfile(fd, dtype = 'uint8')
                return np.reshape(images, (items_n, height, width))
            elif "labels" in file_name and check == 2049:
                return np.fromfile(fd, dtype = 'uint8')
            else:
                raise ValueError("Not a MNIST file: " + path)
    
    
    def get_training_batch(self, batch_size):
        return self._get_batch(self._training_data, self._training_labels, batch_size)
    
    def get_validation_batch(self, batch_size):
        return self._get_batch(self._validation_data, self._validation_labels, batch_size)
    
    def get_test_batch(self, batch_size):
        return self._get_batch(self._test_data, self._test_labels, batch_size)
    
    def _get_batch(self, data, labels, batch_size):
        samples_n = labels.shape[0]
        if batch_size <= 0:
            batch_size = samples_n
        
        random_indices = np.random.choice(samples_n, samples_n, replace = False)
        data = data[random_indices]
        labels = labels[random_indices]
        for i in range(samples_n // batch_size):
            on = i * batch_size
            off = on + batch_size
            yield data[on:off], labels[on:off]
    
    
    def get_sizes(self):
        training_samples_n = self._training_labels.shape[0]
        validation_samples_n = self._validation_labels.shape[0]
        test_samples_n = self._test_labels.shape[0]
        return training_samples_n, validation_samples_n, test_samples_n
    
        
    


In [31]:
mnist = MNIST("/home/laars/uni/WS2017/tensorflow/hw02")
test_batch = mnist.get_test_batch(5)
validation_batch = mnist.get_validation_batch(5)
training_batch = mnist.get_training_batch(100)

mnist.get_training_batch(5)

<generator object MNIST._get_batch at 0x7fb51571b468>

In [61]:
x = tf.placeholder(tf.float32, [None, 784])

bias_vector = tf.zeros([10])
biases = tf.Variable(bias_vector, dtype = tf.float32)

weight_vector = tf.random_normal([784, 10], stddev = 0.000002)
weights =  tf.Variable(weight_vector, dtype = tf.float32)

labels_soft = tf.placeholder(tf.float32, [None, 10])

softmax = tf.nn.softmax_cross_entropy_with_logits(labels = labels_soft, 
                                                  logits = tf.matmul(x, weights) + biases,
                                                 )
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(softmax)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for i in training_batch:
        
        y = np.zeros((100, 10))
        for j, k in y:
            print(j)
            y[j][i[1, j]] = 1
        print(y)
        #batch_xs, batch_ys = mnist.get_training_batch(100)
        session.run(train_step, feed_dict={x: np.reshape(i[0], (100, 784)), 
                                           labels_soft: i[1]})

ValueError: too many values to unpack (expected 2)

In [ ]:
[[0,0,0,0],[0,0,0,0]]
